In [1]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
preprocessing_fp = "preprocessing/combined_filtered/"
BATCH_SIZE = 32
EPOCHS = 500
IMG_SIZE = (224,224)
LABELS = ["female", "male"]

In [27]:
mobilenet = tf.keras.models.load_model('model_tl_best_weights_mobile.h5')
densenet = tf.keras.models.load_model('model_tl_best_weights_dense.h5')
resnet = tf.keras.models.load_model('model_tl_best_weights_res.h5')
all_models = [mobilenet, densenet, resnet]

In [41]:
def make_preds(model_type, gender=None, models=all_models):
    """
    Returns classification report and confusion matrix (sklearn.metrics)
    
    model_type : str
        Either mobile, dense or res
    gender : str
        Either None, male or female to specify the gender. If None it make predictions on both.
    """
    # FEMALE => 0
    # MALE => 1
    
    # Set model
    if (model_type == "mobile"):
        model = all_models[0]
    elif (model_type == "dense"):
        model = all_models[1]
    elif (model_type == "res"):
        model = all_models[2]
    else:
        raise Exception("Sorry, model_type allowed are 'mobile' (MobileNet), 'dense' (DenseNet) \
        or 'res' (ResNet50)")
    
    assert gender in [None, 'male', 'female'], "gender needs to be 'male' or 'female'"
    
    datasets = ["test_ori", "test_masked", "test_glasses", "test_make_up"]
    for i in tqdm(range(len(datasets)), 'Loading test datasets...'):
        data_name = datasets[i]
        y_true = []
        y_pred = []
        male_dir = os.listdir(preprocessing_fp + data_name + "/male")
        female_dir = os.listdir(preprocessing_fp + data_name + "/female")
        
        if gender is None:
            for i in tqdm(range(len(male_dir))):
                fn = male_dir[i]
                img = Image.open(preprocessing_fp + data_name + "/male/" + fn)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(1)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)

            for i in tqdm(range(len(female_dir))):
                fn = female_dir[i]
                img = Image.open(preprocessing_fp + data_name + "/female/" + fn)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(0)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)
        elif gender == 'male':
            for i in tqdm(range(len(male_dir))):
                fn = male_dir[i]
                img = Image.open(preprocessing_fp + data_name + "/male/" + fn)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(1)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)
        elif gender == 'female':
            for i in tqdm(range(len(female_dir))):
                fn = female_dir[i]
                img = Image.open(preprocessing_fp + data_name + "/female/" + i)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(0)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)

        cr = classification_report(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)
        return cr, cm

# Both

In [38]:
# for model in models:
#     make_preds(model)
#     print("\n ================================================= \n")

              precision    recall  f1-score   support

           0       0.80      0.83      0.81      2144
           1       0.79      0.76      0.78      1860

    accuracy                           0.80      4004
   macro avg       0.80      0.80      0.80      4004
weighted avg       0.80      0.80      0.80      4004


[[1774  370]
 [ 438 1422]]
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      2144
           1       0.59      0.66      0.62      1862

    accuracy                           0.63      4006
   macro avg       0.63      0.63      0.63      4006
weighted avg       0.63      0.63      0.63      4006


[[1284  860]
 [ 636 1226]]
              precision    recall  f1-score   support

           0       0.78      0.61      0.69      2144
           1       0.64      0.80      0.71      1860

    accuracy                           0.70      4004
   macro avg       0.71      0.71      0.70      4004
weighted avg      

# Male only

In [42]:
# for model in models:
#     make_preds(model, 'male')
#     print("\n ================================================= \n")

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.76      0.87      1860

    accuracy                           0.76      1860
   macro avg       0.50      0.38      0.43      1860
weighted avg       1.00      0.76      0.87      1860


[[   0    0]
 [ 438 1422]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.66      0.79      1862

    accuracy                           0.66      1862
   macro avg       0.50      0.33      0.40      1862
weighted avg       1.00      0.66      0.79      1862


[[   0    0]
 [ 636 1226]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

...

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.68      0.81      1862

    accuracy                           0.68      1862
   macro avg       0.50      0.34      0.41      1862
weighted avg       1.00      0.68      0.81      1862


[[   0    0]
 [ 591 1271]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.58      0.74      1860

    accuracy                           0.58      1860
   macro avg       0.50      0.29      0.37      1860
weighted avg       1.00      0.58      0.74      1860


[[   0    0]
 [ 774 1086]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.51      0.68      1860

    accuracy                           0.51      1860
   macro avg       0.50      0.25      0.34      1860
weighted avg       1.00      0.51      0.68      1860


[[  0   0]
 [912 948]]




/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

# Female only

In [43]:
# for model in models:
#     make_preds(model, 'female')
#     print("\n ================================================= \n")

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.83      0.91      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.83      2144
   macro avg       0.50      0.41      0.45      2144
weighted avg       1.00      0.83      0.91      2144


[[1774  370]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.60      0.75      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.60      2144
   macro avg       0.50      0.30      0.37      2144
weighted avg       1.00      0.60      0.75      2144


[[1284  860]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

...

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.61      0.76      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.61      2144
   macro avg       0.50      0.31      0.38      2144
weighted avg       1.00      0.61      0.76      2144


[[1312  832]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.88      0.94      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.88      2144
   macro avg       0.50      0.44      0.47      2144
weighted avg       1.00      0.88      0.94      2144


[[1890  254]
 [   0    0]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.95      2144
   macro avg       0.50      0.47      0.49      2144
weighted avg       1.00      0.95      0.97      2144


[[2034  110]
 [   0    0]]




/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le